# 転移学習

In [3]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

- 入力２つ，出力２つ

In [4]:
fashion_mnist=keras.datasets.fashion_mnist
(X_train_full,y_train_full),(X_test,y_test)=fashion_mnist.load_data()
print(X_train_full.shape)
print(y_train_full.shape)
print(X_test.shape)
print(y_test.shape)

4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [17]:
y_test[0:10]

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], dtype=uint8)

In [11]:
X_valid,X_train=X_train_full[:5000]/255.0,X_train_full[5000:]/255.0#勾配降下法をしようするので，ピクセル強度を0~1にスケーリングするために255で割ってる
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]
class_names=["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat","Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
print(class_names[y_train[0]])
print(X_train.shape)
print(X_valid.shape)

Coat
(55000, 28, 28)
(5000, 28, 28)


In [12]:
model = keras.models.Sequential([
keras.layers.Flatten(input_shape=[28, 28]),
keras.layers.Dense(300, activation="relu",name="hidden1"),
keras.layers.Dense(100, activation="relu"),
keras.layers.Dense(10, activation="softmax")
])

In [13]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])

In [14]:
history=model.fit(X_train,y_train,epochs=30,validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 3s 59us/sample - loss: 0.7167 - accuracy: 0.7657 - val_loss: 0.4945 - val_accuracy: 0.8304
Epoch 2/30
55000/55000 [==============================] - 3s 53us/sample - loss: 0.4822 - accuracy: 0.8337 - val_loss: 0.4415 - val_accuracy: 0.8514
Epoch 3/30
55000/55000 [==============================] - 3s 48us/sample - loss: 0.4394 - accuracy: 0.8454 - val_loss: 0.4362 - val_accuracy: 0.8508
Epoch 4/30
55000/55000 [==============================] - 3s 49us/sample - loss: 0.4138 - accuracy: 0.8536 - val_loss: 0.4088 - val_accuracy: 0.8604
Epoch 5/30
55000/55000 [==============================] - 3s 48us/sample - loss: 0.3933 - accuracy: 0.8607 - val_loss: 0.3938 - val_accuracy: 0.8666
Epoch 6/30
55000/55000 [==============================] - 3s 48us/sample - loss: 0.3778 - accuracy: 0.8662 - val_loss: 0.3662 - val_accuracy: 0.8732
Epoch 7/30
55000/55000 [==============================] -

In [18]:
X_new=X_test[:3]
y_proba=model.predict(X_new)
y_proba.round(4)
y_pred=model.predict_classes(X_new)

In [19]:
model.save("For_TransferLearning.h5")

- 学習済みモデルを読み込む

In [26]:
model_A=keras.models.load_model("For_TransferLearning.h5")
"""最終層の一個前までを読み込む"""
model_B_on_A=keras.models.Sequential(model_A.layers[:-1])#.layer[:-1]は最終層の一個前まで
"""最終層(出力層)に新しい層を追加"""
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))

- そのままだと元のモデルも影響を受けちゃうので，クローンをコピーすることで影響を断ち切る

In [27]:
model_A_clone=keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [28]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable=False
model_B_on_A.compile(loss="binary_crossentropy",optimizer="sgd",metrics=["accuracy"])

In [29]:
history=model_B_on_A.fit(X_train,y_train,epochs=4,validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/4
55000/55000 [==============================] - 3s 53us/sample - loss: -10283.8542 - accuracy: 0.0991 - val_loss: -20740.7952 - val_accuracy: 0.1112
Epoch 2/4
55000/55000 [==============================] - 2s 42us/sample - loss: -30866.9723 - accuracy: 0.0990 - val_loss: -41481.4030 - val_accuracy: 0.1112
Epoch 3/4
55000/55000 [==============================] - 2s 42us/sample - loss: -51450.2085 - accuracy: 0.0990 - val_loss: -62221.8716 - val_accuracy: 0.1112
Epoch 4/4
55000/55000 [==============================] - 2s 41us/sample - loss: -72033.2915 - accuracy: 0.0990 - val_loss: -82961.9211 - val_accuracy: 0.1112


- 最終層(出力層)だけ4epoch学習させた．
- 次は全ての層を学習させる

In [32]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable=True
optimizer=keras.optimizers.SGD(lr=0.0001)
model_B_on_A.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history=model_B_on_A.fit(X_train,y_train,epochs=16,validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/16
55000/55000 [==============================] - 3s 56us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 2/16
55000/55000 [==============================] - 3s 54us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 3/16
55000/55000 [==============================] - 3s 48us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 4/16
55000/55000 [==============================] - 3s 47us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 5/16
55000/55000 [==============================] - 3s 49us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 6/16
55000/55000 [==============================] - 3s 49us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 7/16
55000/55000 [==============================] - 3s 50us/sample - loss: nan - accura